# Retrieval-Augmented Generation with Cohere language models on Amazon Bedrock and Weaviate vector database on AWS Market place

The example use case generates targeted advertisements for vacation stay listings based on a target audience. The goal is to use the user query for the target audience (e.g., “family with small children”) to retrieve the most relevant vacation stay listing (e.g., a listing with playgrounds close by) and then to generate an advertisement for the retrieved listing tailored to the target audience.

Note that the following code uses the `v3` Weaviate Python client. A `v4` Weaviate Python client, which uses gRPC under the hood, is being developed (currently in beta as of November 2023). You can find the same RAG pipeline using the new `v4` client in the related GitHub repository.

This notebook should work well with the Data Science 3.0 kernel in SageMaker Studio.

## Dataset Overview
The dataset is available from [Inside AirBnB](http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-09-03/data/listings.csv.gz) and is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).

Download the data and save it in a folder called `data`.

In [ ]:
!wget http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-09-03/data/listings.csv.gz

In [ ]:
!gzip -d listings.csv.gz
!mkdir data
!mv listings.csv ./data

In [31]:
import pandas as pd
import json

# Read CSV file
csv_file = './data/listings.csv'
df = pd.read_csv(csv_file, usecols=['host_name',
                                    'property_type',
                                    'description',
                                    'neighborhood_overview',
                                    ])

df.fillna('Unknown', inplace=True)

display(df.head())

,description,neighborhood_overview,host_name,property_type
0,"Really quiet, spacious and safe, a nice place ...","The neighborhood is very green, quiet, safe an...",Xsjong,Private room in condo
1,Our cool and spacious loft is perfect for a st...,Our neighbourhood is ons of the most beautiful...,J & R,Entire rental unit
2,Room to rent in my houseboat. The room has a p...,It is just outside the Jordan in between the c...,Nicole,Private room in boat
3,"Spacious houseboat in Amsterdam, suitable for ...",The houseboat lies in an area with ± 200 house...,Danielle,Houseboat
4,Royal Bed & Coffee Room with a very comfortabl...,The building is located in Amsterdam centre in...,Marcel,Private room in rental unit


## Prerequisites
To be able to follow along and use any AWS services in the following tutorial, please make sure you have an [AWS account](https://signin.aws.amazon.com/signin?redirect_uri=https%3A%2F%2Fportal.aws.amazon.com%2Fbilling%2Fsignup%2Fresume&client_id=signup).

## Step 1: Enable components of the AI-native technology stack
First, you will need to enable the relevant components discussed in the solution overview in your AWS account. 
First, enable access to the Cohere Command and Embed foundation models available on Amazon Bedrock through the AWS Management Console. Navigate to the Model access page, click on Edit, and select the foundation models of your choice.

Next, set up a Weaviate cluster. In this Notebook, you are using the preview Docker image.
`$AWS_ACCESSKEY` and  `$AWS_ACCESSSECRET` in the docker-compose.yml and then run:

In [ ]:
# docker-compose up -d

## Step 2: Connect to Weaviate
On the SageMaker console, navigate to Notebook instances and create a new notebook instance. 

Then, install the Weaviate client package with the required dependencies:

In [11]:
# pip install weaviate-client

Now, you can connect to your Weaviate instance with the following code. You can find the relevant information as follows:
* Weaviate URL: As you are running Weaviate via Docker Compose, the url is "http://localhost:8080"
* Weaviate API Key: Not needed here
* AWS Access Key: You can retrieve the access keys for your user in the AWS Identity and Access Management (IAM) Console.

In [32]:
import weaviate

client = weaviate.Client(
    url="http://localhost:8080",
    additional_headers={
        "X-AWS-Access-Key": "<AWS-ACCESS-KEY>",
        "X-AWS-Access-Secret": "<AWS-ACCESS-SECRET>"
    }
)

client.get_meta()

{'hostname': 'http://[::]:8080',
 'modules': {'generative-aws': {'documentationHref': 'https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html',
   'name': 'Generative Search - AWS'},
  'text2vec-aws': {'documentationHref': 'https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings',
   'name': 'AWS Module'}},
 'version': '1.22.4'}

## Step 3: Configure the Amazon Bedrock module to enable Cohere models

Next, you will define a data collection (i.e., `class`) called `Listings` to store the listings’ data objects, which is analogous to creating a table in a relational database. In this step, you will configure the relevant modules to enable the usage of Cohere language models hosted on Amazon Bedrock natively from within the Weaviate vector database. The vectorizer (`"text2vec-aws"`) and generative module (` "generative-aws"`) are specified in the data collection definition. Both of these modules take three parameters:
* `"service"`: `"bedrock"` for Amazon Bedrock (Alternatively, `"sagemaker"` for Amazon Sagemaker Jumpstart)
* `"Region"`: The region where your model is deployed
* `"model"`: The foundation model’s name

In [33]:
collection_definition = {
    "class": "Listings",
    "description": "AirBnb Listings",
    "moduleConfig": {
        "text2vec-aws": {
            "service": "bedrock",
            "region": "us-east-1",
            "model": "cohere.embed-english-v3",
            "vectorizeClassName": False,

        },
        "generative-aws": {
            "service": "bedrock",
            "region": "us-east-1",
            "model": "cohere.command-text-v14"
        }
    },
    "vectorizer": "text2vec-aws",
}

## Step 4: Ingest data into the Weaviate vector database 
In this step, you will also define the structure of the data collection by configuring its properties. Aside from the property’s name and data type, you can also configure if only the data object shall be stored or if it shall be stored together with its vector embeddings. In this example, `host_name` and `property_type` are not vectorized.

In [34]:
collection_definition["properties"] = [
    {
        "name": "description",
        "dataType": ["text"],
        "description": "The description of the apartment listing",
    },
    {
        "name": "host_name",
        "dataType": ["text"],
                "description": "The name of the host of the apartment listing",
                "moduleConfig": {
                    "text2vec-aws": {
                        "skip": True,
                        "vectorizePropertyName": False,

                    }
        }
    },
    {
        "name": "neighborhood_overview",
        "dataType": ["text"],
        "description": "The description of  the neighbourhood of the apartment listing",
    },
    {
        "name": "property_type",
                "dataType": ["text"],
                "description": "The type of property of the listing",
                "moduleConfig": {
                    "text2vec-aws": {
                        "skip": True,
                        "vectorizePropertyName": False,
                    }
                }
    },
]

Run the following code to create the collection in your Weaviate instance.

In [35]:
if client.schema.exists("Listings"):
    client.schema.delete_class("Listings")

client.schema.create_class(collection_definition)

client.schema.get("Listings")

{'class': 'Listings',
 'description': 'AirBnb Listings',
 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
 'moduleConfig': {'generative-aws': {'model': 'cohere.command-text-v14',
   'region': 'us-east-1',
   'service': 'bedrock'},
  'text2vec-aws': {'model': 'cohere.embed-english-v3',
   'region': 'us-east-1',
   'service': 'bedrock',
   'vectorizeClassName': False}},
 'multiTenancyConfig': {'enabled': False},
 'properties': [{'dataType': ['text'],
   'description': 'The description of the apartment listing',
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'text2vec-aws': {'skip': False,
     'vectorizePropertyName': False}},
   'name': 'description',
   'tokenization': 'word'},
  {'dataType': ['text'],
   'description': 'The name of the host of the apartment listing',
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'text

You can now add objects to Weaviate. You will be using a batch import process for maximum efficiency. Run the code below to import data. During the import, Weaviate will use the defined vectorizer to create a vector embedding for each object. The following code loads objects initializes a batch process, and adds objects to the target collection one by one.

In [36]:
from weaviate.util import generate_uuid5

client.batch.configure(batch_size=100)  # Configure batch

# Initialize batch process
with client.batch as batch:
    for _, row in df.iterrows():
        listing_object = {
            "description": row["description"],
            "host_name": row["host_name"],
            "neighborhood_overview": row["neighborhood_overview"],
            "property_type": row["property_type"],
        }
        batch.add_data_object(
            class_name="Listings",
            data_object=listing_object,
            uuid=generate_uuid5(listing_object)
        )

client.query.aggregate("Listings").with_meta_count().do()

{'data': {'Aggregate': {'Listings': [{'meta': {'count': 100}}]}}}

## Step 5: Retrieval-Augmented Generation to generate targeted advertisements

Finally, you can build a RAG pipeline by implementing a generative search query on your Weaviate instance. For this, you will first define a prompt template in the form of an f-string that can take in the user query (`{target_audience}`) directly and the additional context (`{{host_name}}`, `{{property_type}}`, `{{description}}`, `{{neighborhood_overview}}`) from the vector database at runtime.

Next, you will run a generative search query. This prompts the defined generative model with a prompt that is comprised of the user query as well as the retrieved data. The following query retrieves one listing object (`.with_limit(1)`) from the `Listings` collection that is most similar to the user query (`.with_near_text({"concepts": target_audience})`). Then the user query (`target_audience`) and the retrieved listings properties (`["description", "neighborhood", "host_name", "property_type"]`) are fed into the prompt template.

Below, you can see that the results for the `target_audience = “Family with small children”`.

In [38]:
result = generate_targeted_ad("Families with young children")
# print(json.dumps(result["data"]["Get"]["Listings"], indent=4))
print(result["data"]["Get"]["Listings"][0]
      ["_additional"]["generate"]["singleResult"])

Looking for a fun-filled and comfortable vacation home for your next trip to Amsterdam? Look no further than this incredible 175 square meter loft, located just a stone's throw away from the Vondelpark! This home is especially perfect for families traveling with children, and is equipped with everything you might need for a relaxing and enjoyable stay. 

The space itself is uniquely designed as a converted garage, and boasts a spacious and open layout. On the ground floor, you'll find a fully equipped


Here is another example for an elderly couple.

In [39]:
result = generate_targeted_ad("Elderly couple")
# print(json.dumps(result["data"]["Get"]["Listings"], indent=4))
print(result["data"]["Get"]["Listings"][0]
      ["_additional"]["generate"]["singleResult"])

Experience the ultimate in romance and relaxation with a stay at the stylish and historic M.S. Luctor, the perfect houseboat for the perfect couple. Imagine waking up in a luxurious bedroom, with sunlight dancing on the water as you enjoy a delicious breakfast on the jetty, perhaps sharing a piece with the resident Greek turtle. 

Explore the city by bicycle or Canadian canoe, with two bikes and a canoe included with your stay. Whether you choose to venture out to discover the breathtaking view or
